In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
#References from documentation
trans = torchvision.transforms.Compose([
    torchvision.transforms.RandAugment(),
    torchvision.transforms.Resize(size =(512,512)),
    ToTensor()
])
batchS = 70
data = torchvision.datasets.ImageFolder('../input/wild-animals/Wild_animals_512',transform = trans)
train_data,test_data = torch.utils.data.random_split(data,[round(data.__len__()*0.6),round(data.__len__()*0.4)])

#valid_data,testy_data = torch.utils.data.random_split(test_data,[round(test_data.__len__()*0.5),round(test_data.__len__()*0.5)])
valid_data,testy_data = torch.utils.data.random_split(test_data,[345,344])
train_load = DataLoader(train_data,shuffle = True,batch_size= batchS,drop_last=True)
valid_load = DataLoader(valid_data,shuffle = True,batch_size= batchS,drop_last = True)
test_load = DataLoader(testy_data,shuffle = True,batch_size= batchS,drop_last = True)
# print(len(test_load.dataset))

print('done')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

done


In [10]:
#Neural Network referenced from  https://pytorch.org/tutorials/beginner/examples_nn/polynomial_module.html
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

#Initial Model code referenced from https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
#Code from the 'Define the Network' section
#Experiment 1
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        #Feature extraction
        self.conv= nn.Conv2d(3,64,9)
        self.conv2 = nn.Conv2d(64,128,5)
        self.conv3 = nn.Conv2d(128,256,5)
        self.conv4 = nn.Conv2d(256,512,5)
        self.bat2d1 = nn.BatchNorm2d(64)
        self.bat2d2 = nn.BatchNorm2d(128)
        self.bat2d3 = nn.BatchNorm2d(256)
        self.bat2d4 = nn.BatchNorm2d(512)
        self.bat1d = nn.BatchNorm1d(196)
        self.dropout = nn.Dropout(p= 0.5)

        
        #Classification layer
        self.lin = nn.Linear(921600,196)
        self.lin2 = nn.Linear(196,6)
        
    def forward(self,x):
        #First two lines are referenced
        x = F.max_pool2d(F.relu(self.conv(x)),(2,2))
        x=self.bat2d1(x)
        self.dropout(x)
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=self.bat2d2(x)
        self.dropout(x)
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        x=self.bat2d3(x)
        self.dropout(x)
        x=self.bat2d4(x)
        self.dropout(x)
        #print(x.shape)
        #Flattening tensors
        x = torch.flatten(x,1)
#         print(x.shape)
        x = F.relu(self.lin(x))
        self.bat1d(x)
        x = F.softmax(self.lin2(x),dim = 1)
        return x
    
model = Model()
device = torch.device("cuda:0")
#model.to(torch.device("cuda:0"))
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.001,weight_decay=0.000001)
lossfn = nn.CrossEntropyLoss()
lossx = 0
lossxt = 0

#count = []
lossgraph = []
testgraph = []
#Code referenced from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
#I extracted data from the DataLoaders following the tutorial above
#I used two for loops and called the optimizer and loss functions in a similar way
#From the 'Define the network' section of the webpage.
for epoch in range(100):

    
    for i,data in enumerate(train_load, 0):
        lossx = 0
        pics, label = data
        pics, label = pics.cuda(), label.cuda()
        #print(pics.shape)
        optimizer.zero_grad()
        output = model(pics)
        #reshaping label
        #label = torch.tensor(label)
        labely = torch.zeros(batchS,6)
        for j in range(batchS):
            labely[j][label[j]] = 1
        #print(output)
        labely = labely.cuda()
        loss = lossfn(output,labely)
        #lossgraph.append(loss)
        loss.backward()
        lossx += loss
        #print(output)
        optimizer.step()
        #graphing loss
        
        #print(len(lossgraph),epoch,i)
    lossgraph.append(lossx.item())

    testingloss = nn.CrossEntropyLoss()
#     classes = {0:'Motorbikes',1:'Airplanes',2:'Schooner'}
    total = 0
    correct = 0
    for i, data in enumerate(valid_load,0):
        lossxt = 0
        pics,label = data
        pics, label = pics.cuda(), label.cuda()
        #print(pics.shape)
        output = model(pics)
        labely = torch.zeros(batchS,6)
        for j in range(batchS):
            labely[j][label[j]] = 1
            #print(output)
        labely = labely.cuda()
        loss = testingloss(output,labely)
        lossxt += loss.item()
        #print(output)
        maxele,maxindx = torch.max(output,1)
        for j in range(batchS):
            #print('Prediction: {0}, Answer: {1}'.format(classes[maxindx[j].item()],classes[label[j].item()]))
            #print(maxindx[j].item(),label[j].item())
            if(torch.eq(maxindx[j],label[j])):
                correct = correct +1
            total = total+1
    #print(correct,total)
    
    #plt.legend('Training set','Validation set')
    accuracy = (correct/total)*100
    #print('Accuracy is {0:.2f}%'.format(accuracy))
    testgraph.append(lossxt)

        #plt.ylim(4e-1,7e-1)
#plt.yscale('log')
#plt.figure(figsize = (1,1))
#print(testgraph)

#plt.yscale('log')



torch.cuda.empty_cache()
        
        
    



    
#for epoch,data in enumerate(train_load,0):
    #inputy, label= data
    #forward(inputy)
#ask how to get gradient of linear stuff and how to determine convolutions

KeyboardInterrupt: 

In [ ]:
plt.yscale('log')
plt.plot(testgraph,'r-')
plt.plot(lossgraph)

In [ ]:
#validation cell
#Code referenced from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
#I extracted data from the DataLoaders following the tutorial above
#I used two for loops and called the optimizer and loss functions in a similar way
#From the 'Define the network' section of the webpage.
'''

'''
#testgraph = []
#testingloss = nn.CrossEntropyLoss()
classes = {0:'Motorbikes',1:'Airplanes',2:'Schooner'}
total = 0
correct = 0
cl = []
pl = []
for i, data in enumerate(valid_load,0):
    lossxt = 0
    pics,label = data
    pics, label = pics.cuda(), label.cuda()
    #print(pics.shape)
    output = model(pics)
    labely = torch.zeros(batchS,6)
    for j in range(batchS):
        labely[j][label[j]] = 1
        #print(output)
    labely = labely.cuda()
    #loss = testingloss(output,labely)
    #lossxt += loss.item()
    #print(output)
    maxele,maxindx = torch.max(output,1)
    #testgraph.append(lossxt)
    for j in range(batchS):
        #print('Prediction: {0}, Answer: {1}'.format(classes[maxindx[j].item()],classes[label[j].item()]))
        #print(maxindx[j].item(),label[j].item())
        cl.append(label[j].item())
        pl.append(maxindx[j].item())
        if(torch.eq(maxindx[j],label[j])):
            correct = correct +1
        total = total+1
print(correct,total)
#plt.plot(testgraph)
#plt.legend('Training set','Validation set')
accuracy = (correct/total)*100
fscore = f1_score(cl,pl,average='macro')
print("F1 score is {0:.2f}".format(fscore))
print('Accuracy is {0:.2f}%'.format(accuracy))
    

In [ ]:
#Testing data
#Code referenced from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
#I extracted data from the DataLoaders following the tutorial above
#I used two for loops and called the optimizer and loss functions in a similar way.
#From the 'Define the network' section of the webpage.
total = 0
correct =0
cl2 = []
pl2 = []
for i2, data2 in enumerate(test_load,0):
    #lossxt = 0
    pics,label = data2
    pics, label = pics.cuda(), label.cuda()
    #print(pics.shape)
    output = model(pics)
    labely = torch.zeros(batchS,3)
    for j in range(batchS):
        labely[j][label[j]] = 1
        #print(output)
    labely = labely.cuda()
    loss = testingloss(output,labely)
    #lossxt += loss
    #print(output)
    maxele,maxindx = torch.max(output,1)
    #testgraph.append(lossxt)
    for j in range(batchS):
        #print('Prediction: {0}, Answer: {1}'.format(classes[maxindx[j].item()],classes[label[j].item()]))
        #print(maxindx[j].item(),label[j].item())
        cl2.append(label[j].item())
        pl2.append(maxindx[j].item())
        if(torch.eq(maxindx[j],label[j])):
            correct = correct +1
        total = total+1
print(correct,total)
#plt.plot(testgraph)
accuracy = (correct/total)*100
fscore = f1_score(cl2,pl2,average='macro')
print("F1 score is {0:.2f}".format(fscore))
print('Accuracy is {0:.2f}%'.format(accuracy))


# KNN model
I am utilizing the knn method of classification on these images by first putting on the images and their labels into seperate list. I will then put these list into sklearn's knn function.

In [10]:
#https://medium.com/swlh/image-classification-with-k-nearest-neighbours-51b3a289280
from pathlib import Path
import numpy as np
from keras.preprocessing import image
from skimage.io import imread
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import cv2
import skimage.io
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
trans = torchvision.transforms.Compose([
    torchvision.transforms.RandAugment(),
    torchvision.transforms.Resize(size =(32,32)),
    ToTensor()
])
il = np.empty((1723,1))
il = []
labe = [] 
validlabe = []
index = np.random.choice(1723,1723)
print(len(index))
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	
	cv2.normalize(hist,hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()
p = Path('../input/wild-animals/Wild_animals_512')
parameters = {'n_neighbors':[100,500], 'weights':('uniform','distance'),'leaf_size':[30,100],'p':(1,2)}
folders = [d for d in p.iterdir() if d.is_dir()]
classes = [d.name for d in folders]
counter =0
for i,direc in enumerate(folders):
    for file in direc.iterdir():
        img = cv2.imread(str(file))
        cv2.resize(img,(32,32))
        tot =  extract_color_histogram(img)
#         histogramr, bin_edges = np.histogram(img[:,:0], bins=256, range=(0, 1))
#         histogramg, bin_edges = np.histogram(img[:,:1], bins=256, range=(0, 1))
#         histogramb, bin_edges = np.histogram(img[:,:0], bins=256, range=(0, 1))
#         tot= np.concatenate((histogramr,histogramg,histogramb))
#         blue,red,green = cv2.split(img)
        
#         pca_Blue = PCA(n_components = 20)
#         new_b = pca_Blue.fit_transform(blue)
#         np.array(new_b)
# #         normalize(new_b)
#         new_b = np.ndarray.flatten(new_b)
        
#         pca_R = PCA(n_components = 20)
#         new_r = pca_R.fit_transform(red)
#         np.array(new_r)
# #         normalize(new_r)
#         new_r = np.ndarray.flatten(new_r)
        
#         pca_G = PCA(n_components = 20)
#         new_g = pca_G.fit_transform(green)
# #         np.array(new_g)
#         normalize(new_g)
#         new_g = np.ndarray.flatten(new_g)
#         con = np.concatenate([new_r,new_g,new_b])
#         print(np.shape(con))
        il.append(tot)
        labe.append(str(direc))
#     print(il)
# normalize(il)
# print(np.shape(il))
itest = []
ilabel = []
for i in range(1723):
    itest.append(il[index[i]])
    ilabel.append(labe[index[i]])
        
data2 = torchvision.datasets.ImageFolder('../input/wild-animals/Wild_animals_512',transform = trans)
train_data2,test_data2 = torch.utils.data.random_split(data2,[round(data2.__len__()*0.6),round(data2.__len__()*0.4)])
valid_data2,testy_data2 = torch.utils.data.random_split(test_data2,[345,344])
images  = []
labs = []
validImages = []
validlabs = []
featureExtractor = nn.Sequential(

        nn.Conv2d(3,64,9),
        nn.ReLU()
)
featureExtractor.to(torch.device('cuda:0'))
for i in range(len(train_data2)):
    pics, label = train_data2[i]
#     pics = pics.cuda()
#     pics = featureExtractor(pics)
#     images.append(pics.flatten().numpy())
    labs.append(label)
    
# print(len(valid_data2))
for j in range(len(valid_data2)):
    pics,label = valid_data2[j]
#     pics = pics.cuda()
#     pics = featureExtractor(pics)
#     validImages.append(pics.flatten().numpy())
    validlabs.append(label)
print('Doing knn now')
print(len(il))
correct = 0
total = 0
modelKnn = KNeighborsClassifier(n_neighbors = 200)
grid = GridSearchCV(modelKnn,parameters)
print(np.shape(itest[:1034]),np.shape(np.array(ilabel)),np.shape(itest[1035:1380]))
modelKnn.fit(itest[:1034],ilabel[:1034])
for i in range(len(validlabs)):
    pred = modelKnn.predict(itest[1035+i].reshape(1,-1))
    if pred == ilabel[1035+i]:
        print('Yay')
        correct+=1
    total += 1
    print(f'Actual is {ilabel[1035+i]} and predicted is {pred}')
accuracy = (correct/total)*100
print(accuracy)
    




    

1723


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Doing knn now
1723
(1034, 512) (1723,) (345, 512)
Actual is ../input/wild-animals/Wild_animals_512/tiger-resize-512 and predicted is ['../input/wild-animals/Wild_animals_512/hyena-resize-512']
Actual is ../input/wild-animals/Wild_animals_512/cheetah-resize-512 and predicted is ['../input/wild-animals/Wild_animals_512/hyena-resize-512']
Actual is ../input/wild-animals/Wild_animals_512/cheetah-resize-512 and predicted is ['../input/wild-animals/Wild_animals_512/tiger-resize-512']
Yay
Actual is ../input/wild-animals/Wild_animals_512/hyena-resize-512 and predicted is ['../input/wild-animals/Wild_animals_512/hyena-resize-512']
Actual is ../input/wild-animals/Wild_animals_512/lion-resize-512 and predicted is ['../input/wild-animals/Wild_animals_512/tiger-resize-512']
Yay
Actual is ../input/wild-animals/Wild_animals_512/wolf-resize-512 and predicted is ['../input/wild-animals/Wild_animals_512/wolf-resize-512']
Actual is ../input/wild-animals/Wild_animals_512/tiger-resize-512 and predicted is 

# Decision Tree Classifier

In [ ]:
%matplotlib inline
#22% for raw pixels
# 30 % after convolutions
from sklearn.ensemble import RandomForestClassifier
from matplotlib.pyplot import imshow
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
images = []
validImages = []
classes = []
labs = []
validlabs = []
featureExtractor = nn.Sequential(

        nn.Conv2d(3,64,9),
        nn.ReLU(),
#         nn.BatchNorm2d(64),
        nn.MaxPool2d(2),
        nn.Conv2d(64,128,3),
        nn.ReLU(),
#         nn.BatchNorm2d(128),
        nn.MaxPool2d(2),
        nn.Conv2d(128,256,3),
        nn.ReLU(),
#         nn.BatchNorm2d(),
        nn.MaxPool2d(2)
        
)
path = '../input/wild-animals/Wild_animals_512'
trans = torchvision.transforms.Compose([
    torchvision.transforms.RandAugment(),
    torchvision.transforms.Resize(size =(32,32)),
    ToTensor()
])
data2 = torchvision.datasets.ImageFolder('../input/wild-animals/Wild_animals_512',transform = trans)
train_data2,test_data2 = torch.utils.data.random_split(data2,[round(data2.__len__()*0.6),round(data2.__len__()*0.4)])
valid_data2,testy_data2 = torch.utils.data.random_split(test_data2,[345,344])
for i in range(len(train_data2)):
    pics, label = train_data2[i]
#     pics = pics.cuda()
#     pics = featureExtractor(pics)
    images.append(featureExtractor(pics).detach().flatten().numpy())
    labs.append(label)
    
# print(len(valid_data2))
for j in range(len(valid_data2)):
    pics,label = valid_data2[j]
#     pics = pics.cuda()
#     pics = featureExtractor(pics)
    validImages.append(featureExtractor(pics).detach().flatten().numpy())
    validlabs.append(label)
# for direc in os.listdir(path):
#     for file in os.listdir(os.path.join(path,direc)):
        
#         i = cv2.imread(os.path.join(path,direc,file),cv2.COLOR_BGR2RGB)
#         i = cv2.normalize(i,i)
#         i = cv2.resize(i,(32,32))
        
#         np.array(i)
#         i = np.ndarray.flatten(i)
        
#         i = np.ndarray.tolist(i)
# #         print(np.shape(i))
#         images.append(i)
#         classes.append(direc)
# print(images)
# print(classes) 
# temp = list(zip(images,classes))
# np.random.shuffle(temp)
# images,classes = zip(*temp)
# np.random.shuffle(images)
clf = RandomForestClassifier()
clf.fit(images,labs)
print(clf.score(validImages,validlabs))
correct = 0
total = 0
for i in range(len(validlabs)):
    pred = clf.predict(np.array(validImages[i]).reshape(1,-1))
    if pred == validlabs[i]:
        correct +=1
    total += 1
    print('Actual: {0}, Predicted: {1}'.format(validlabs[i],pred))
accuracy = (correct/total)*100
print(accuracy)
    